<a href="https://colab.research.google.com/github/modouseck/first-repo/blob/main/checkpoint_web_sccraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def get_html(url):
    """
    Récupère et analyse le contenu HTML d'une page Wikipédia.
    :param url:https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Accueil_principal
    :return: BeautifulSoup object
    """
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')


def get_title(soup):
    """
    Extrait le titre de l'article Wikipédia.
    :param soup: BeautifulSoup object
    :return: titre de l'article (str)
    """
    title_tag = soup.find('h1', id='firstHeading')
    return title_tag.get_text(strip=True) if title_tag else None


def get_text_sections(soup):
    """
    Extrait le texte de l'article organisé par sections et paragraphes.
    :param soup: BeautifulSoup object
    :return: dict mapping section titles to lists of paragraph textes
    """
    content = soup.find('div', class_='mw-parser-output')
    sections = {}
    current_header = 'Introduction'
    sections[current_header] = []

    for elem in content.find_all(['h2', 'h3', 'p'], recursive=False):
        if elem.name in ['h2', 'h3']:
            # retirer l'ancre [modifier]
            header_text = elem.get_text().replace('[modifier]', '').strip()
            current_header = header_text
            sections[current_header] = []
        elif elem.name == 'p':
            text = elem.get_text(strip=True)
            if text:
                sections[current_header].append(text)
    return sections


def get_internal_links(soup, base_url='https://fr.wikipedia.org'):
    """
    Collecte tous les liens internes vers d'autres pages Wikipédia.
    :param soup: BeautifulSoup object
    :param base_url: domaine de base pour compléter les URLs relatives
    :return: liste d'URLs internes
    """
    links = set()
    for a in soup.find_all('a', href=True):
        href = a['href']
        # filtrer les liens vers d'autres articles
        if href.startswith('/wiki/') and ':' not in href:
            full_url = urljoin(base_url, href)
            links.add(full_url)
    return list(links)


def parse_wikipedia(url):
    """
    Fonction consolidée qui prend une URL Wikipédia et retourne:
      - titre
      - sections de texte
      - liens internes
    :param url: URL de la page Wikipédia
    :return: dict contenant 'title', 'sections', 'links'
    """
    soup = get_html(url)
    title = get_title(soup)
    sections = get_text_sections(soup)
    links = get_internal_links(soup)
    return {
        'url': url,
        'title': title,
        'sections': sections,
        'links': links
    }


if __name__ == '__main__':
    # Exemple de test sur la page Wikipédia 'Python (langage)'
    test_url = 'https://fr.wikipedia.org/wiki/Python_(langage)'
    result = parse_wikipedia(test_url)
    print(f"Titre de l'article: {result['title']}\n")
    print("Sections et paragraphes:")
    for section, paras in result['sections'].items():
        print(f"\n== {section} ==")
        for p in paras[:3]:  # n'afficher que les 3 premiers paragraphes de chaque section
            print(f"- {p}")
    print(f"\nNombre de liens internes trouvés: {len(result['links'])}")


Titre de l'article: Python (langage)

Sections et paragraphes:

== Introduction ==
- Python(prononcé/pi.tɔ̃/) est unlangage de programmationinterprété,multiparadigmeetmultiplateformes. Il favorise laprogrammation impérativestructurée,fonctionnelleetorientée objet. Il est doté d'untypage dynamiquefort, d'une gestion automatique de lamémoireparramasse-mietteset d'unsystème de gestion d'exceptions; il est ainsi similaire àPerl,Ruby,Scheme,SmalltalketTcl.
- Le langage Python est placé sous unelicence libreproche de lalicence BSDet fonctionne sur la plupart desplateformes informatiques, dessmartphonesauxordinateurs centraux, deWindowsàUnixavec notammentGNU/Linuxen passant parmacOS, ou encoreAndroid,iOS, et peut aussi être traduit enJavaou.NET. Il est conçu pour optimiser la productivité des programmeurs en offrant des outils dehaut niveauet unesyntaxesimple à utiliser.
- Il est également apprécié par certainspédagoguesqui y trouvent unlangageoù la syntaxe, clairement séparée des mécanismes 